In [2]:
%matplotlib inline
#import mpld3
#mpld3.enable_notebook()
# Default modules
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import json
import ast
import xarray as xr
import dask
import pandas as pd
from netCDF4 import Dataset
from datetime import datetime, timedelta
import jdcal
from scipy import signal
from astropy.io import ascii
import pytz
# OS interaction
import sys
import os
# Nic's modules
#from Precip_QC_methods import Process_Geonor
#from Met_conversions import RH_to_Q

In [2]:
# Directories
main_dir   = os.path.normpath('F:\Work\e\Data\Obs\Canada_Project_Sites\CRHO_Sites\Raw')
dir_in     = main_dir + '\ASCII'
dir_out = main_dir + '\\netcdf'

In [3]:
input_format = 'CRHO_TELM'


In [4]:
if input_format == 'CRHO_TELM':
    header = 4 # Header lines
    # time zone variables
    #tz_in = pytz.timezone('Canada/Mountain')
    tz_in = pytz.timezone('Etc/GMT-6')

In [5]:
# Get file in info
os.chdir(dir_in)
content = os.listdir(os.getcwd())
num_files = len([name for name in os.listdir('.') if os.path.isfile(name)])

In [ ]:
# Read in each file
for cfile in content:
    # Get current station name
    csta_name = cfile[0:3]
    print('Processing ' + csta_name)
    
    # Import data to pandas dataframe
    dat = ascii.read(cfile,header_start=1,data_start=4,delimiter=',')
    datain = pd.DataFrame(dat.as_array())
    
    # Make TIMESTAMP the index
    datain['TIMESTAMP'] = datain['TIMESTAMP'].astype('datetime64[ns]')
    datain = datain.set_index('TIMESTAMP')
    #datain.rename({'TIMESTAMP':'time'})
    
    # Set time zone
    datain.index = datain.index.tz_localize(tz_in)
    
    # Import header info
    headerinfo = pd.read_csv(cfile,nrows=2,skiprows=1)
    
    # Convert to xray
    ds = xr.Dataset.from_dataframe(datain)
    
    # Add new site dimension and make record
    
    # Add Coords
    print('need to get correct site location info')
    ds.coords['LAT'] = 39.0
    ds.coords['LON'] = -120.0
    #ds.coords['sta_name'] = csta_name
    
    # Add variable attributes (units and sample period)
    for cvar in ds.data_vars:
        ds.get(cvar).attrs['units'] = headerinfo.loc[0,[cvar]].values[0]
        ds.get(cvar).attrs['sample'] = headerinfo.loc[1,[cvar]].values[0]
        
    # Change variable names?
    
    # Export to netcdf
    cfileout = os.path.join(dir_out,csta_name + '.nc')
    ds.to_netcdf(cfileout,format='netcdf4') #,encoding={'AirTemp_Avg': {'dtype': 'int16', 'scale_factor': 0.1, '_FillValue': -9999}})

In [6]:
# OPen and combine multiple files
os.chdir(dir_out)
ds_ALL = xr.open_mfdataset('BNS.nc',chunks=10,concat_dim='Site')

TypeError: argument of type 'numpy.float64' is not iterable